# ***1. Importing Essential Libraries***

In [0]:
import pandas as pd # for data processing
import folium #for creating maps
import requests #for retreiving Information
from geopy.geocoders import Nominatim #converting address to cordinates
from pandas.io.json import json_normalize #converting json to DataFrame 

# 2. Saving Creditials of API using (Ex yelp,google places) ( personal to user)

In [0]:
## This is not personal and can not be made private
## get the api key from the provideres

# ***3. Reading the address from user***

In [0]:
address = input("Enter address : ")
pincode = input("Enter postal code : ")

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
if(location == None):
    location = geolocator.geocode(pincode)
    if(location == None):
        print("Address not Found............")
    else:
        print(location)
else:
    print(location)
    truth = input("Is this address nearly accurate.....(y/n) : ")
    if truth == 'n' or truth =='N':
        print("searching based on pincode .........")
        location = geolocator.geocode(pincode)
        if(location == None):
            print("No location found based on pincode..")
        else:    
            print(location)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

Enter address : VR Bengaluru
Enter postal code : 568646
VR Bengaluru, 3rd Main Road, Justice Bheemiah Layout, Devasandra, Mahadevapura Zone, Bengaluru, Bangalore East, Bangalore Urban, Karnataka, 560048, India
Is this address nearly accurate.....(y/n) : y
12.9963426 77.6954215


# ***4. Retreiving the Data***


In [0]:
radius = input("Enter the radius for searching : ")
Limit = input("enter the Limit for Results to display : ")
url = #url provided by your providers
url

In [0]:
results = requests.get(url).json()
results
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-53f1cc33498e0e419065fefb-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/creperie_',
     'suffix': '.png'},
    'id': '52e81612bcbc57f1066b79f2',
    'name': 'Creperie',
    'pluralName': 'Creperies',
    'primary': True,
    'shortName': 'Creperie'}],
  'id': '53f1cc33498e0e419065fefb',
  'location': {'address': 'Phoenix Market City',
   'cc': 'IN',
   'country': 'India',
   'distance': 182,
   'formattedAddress': ['Phoenix Market City', 'India'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 12.997361986513575,
     'lng': 77.69674382065826}],
   'lat': 12.997361986513575,
   'lng': 77.69674382065826},
  'name': 'BelgYum',
  'photos': {'count': 0, 'groups': []}}}

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [0]:
dataframe = json_normalize(items) # flatten JSON
dataframe
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
nearby = dataframe.loc[:, filtered_columns]

# filter the category for each row
nearby['venue.categories'] = nearby.apply(get_category_type, axis=1)

# clean columns
nearby.columns = [col.split('.')[-1] for col in nearby.columns]

#replce NaN values with Not found in address
nearby['address'] = nearby['address'].fillna("Not found")
nearby.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,city,state,crossStreet,postalCode,neighborhood,id
0,BelgYum,Creperie,Phoenix Market City,12.997362,77.696744,"[{'label': 'display', 'lat': 12.99736198651357...",182,IN,India,"[Phoenix Market City, India]",NaN,NaN,NaN,NaN,NaN,53f1cc33498e0e419065fefb
1,Whitefield Social,Lounge,Not found,12.997343,77.696288,"[{'label': 'display', 'lat': 12.99734264495814...",145,IN,India,"[Bangalore, Karnātaka, India]",Bangalore,Karnātaka,NaN,NaN,NaN,5662f272498eac2dc526e83a
2,PVR IMAX VR Bangalore,Movie Theater,Not found,12.996194,77.695363,"[{'label': 'display', 'lat': 12.99619430368990...",17,IN,India,"[Bangalore, Karnātaka, India]",Bangalore,Karnātaka,NaN,NaN,NaN,568653a2498e14934d6314a7
3,VR Bengaluru,Shopping Mall,Not found,12.996095,77.695393,"[{'label': 'display', 'lat': 12.99609462496371...",27,IN,India,"[Bangalore, Karnātaka, India]",Bangalore,Karnātaka,NaN,NaN,NaN,56224801498ee4a64eaffcaa
4,Phoenix Marketcity,Shopping Mall,Whitefield Main Rd,12.997418,77.696559,"[{'label': 'display', 'lat': 12.9974177, 'lng'...",171,IN,India,"[Whitefield Main Rd (Mahadevapura), Krishnaraj...",Krishnarajapura,Karnātaka,Mahadevapura,560048,NaN,4ea2b2e7722ec75ade643f30
5,Toscano,Italian Restaurant,Not found,12.995942,77.695795,"[{'label': 'display', 'lat': 12.99594191336187...",60,IN,India,"[Bangalore, Karnātaka, India]",Bangalore,Karnātaka,NaN,NaN,NaN,57247410498e4345510ce8b7
6,Hamleys,Toy / Game Store,Phoenix Market City,12.997499,77.696655,"[{'label': 'display', 'lat': 12.99749916089661...",185,IN,India,"[Phoenix Market City (Whitefield Rd), Bangalor...",Bangalore,Karnātaka,Whitefield Rd,560048,NaN,5094dbc2e4b087ea434ca2ba
7,PVR Market City,Multiplex,The Phoenix Market City Mall,12.997531,77.695813,"[{'label': 'display', 'lat': 12.99753093023634...",138,IN,India,[The Phoenix Market City Mall (Whitefield Main...,Bangalore,Karnātaka,"Whitefield Main Rd, Mahadevapura Post",560048,NaN,51c1ddc4498e80f222edbfcb
8,Decathlon Sports India (DSI) HQ,Office,"Xylem Building, ITPL Main Road, Whitefield",12.994772,77.701066,"[{'label': 'display', 'lat': 12.99477232652780...",636,IN,India,"[Xylem Building, ITPL Main Road, Whitefield, B...",Bangalore,Karnātaka,NaN,NaN,NaN,4ffd7b2ae4b0c07a15dc42b3
9,Decathlon,Sporting Goods Shop,Mahadevapura,12.994830,77.701010,"[{'label': 'display', 'lat': 12.99482994819446...",629,IN,India,"[Mahadevapura (ITPL Main Road), Bangalore 5600...",Bangalore,Karnātaka,ITPL Main Road,560048,NaN,511f882be4b07e642792e247


In [0]:
nearby.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Windmills Craftworks,Brewery,#331,"Road #5B, EPIP Zone, Whitefield",12.982460,77.721861,"[{'label': 'display', 'lat': 12.98246025089549...",1266,560048,IN,Bangalore,Karnātaka,India,"[#331 (Road #5B, EPIP Zone, Whitefield), Banga...",5056abd4e4b07ba3e77d061b
1,Masala Chai,Café,Brigade Metropolis,NaN,12.992018,77.702983,"[{'label': 'display', 'lat': 12.99201795311745...",1361,NaN,IN,NaN,Karnātaka,India,"[Brigade Metropolis, Karnātaka, India]",52d7b826498ec7118d306846
2,Decathlon Sports India (DSI) HQ,Office,"Xylem Building, ITPL Main Road, Whitefield",NaN,12.994772,77.701066,"[{'label': 'display', 'lat': 12.99477232652780...",1599,NaN,IN,Bangalore,Karnātaka,India,"[Xylem Building, ITPL Main Road, Whitefield, B...",4ffd7b2ae4b0c07a15dc42b3
3,Decathlon,Sporting Goods Shop,Mahadevapura,ITPL Main Road,12.994830,77.701010,"[{'label': 'display', 'lat': 12.99482994819446...",1606,560048,IN,Bangalore,Karnātaka,India,"[Mahadevapura (ITPL Main Road), Bangalore 5600...",511f882be4b07e642792e247
4,Forum Shantiniketan Mall,Shopping Mall,Prestige Shantiniketan,NaN,12.989626,77.728220,"[{'label': 'display', 'lat': 12.98962565178195...",1401,560048,IN,Bengaluru South,Karnātaka,India,"[Prestige Shantiniketan, Bengaluru South 56004...",5b6d42b8b54618002c6bde07
5,Biergarten,Brewery,Not found,NaN,12.982449,77.708620,"[{'label': 'display', 'lat': 12.98244927136445...",1303,NaN,IN,NaN,NaN,India,[India],579e2035498e20f50f6b6108
6,M Cafe (Marriott),Restaurant,Not found,NaN,12.979192,77.728240,"[{'label': 'display', 'lat': 12.97919153643412...",1985,NaN,IN,NaN,NaN,India,[India],5130dc71e4b00e1a5a3c2ac3
7,Power Play Sports,Soccer Field,19/1,"7th Cross Road, Whitefield",12.975183,77.712850,"[{'label': 'display', 'lat': 12.97518295111998...",1897,560066,IN,Bangalore,Karnātaka,India,"[19/1 (7th Cross Road, Whitefield), Bangalore ...",4feb49a1e4b0f878206ed3c4
8,Central Park Clubhouse,Gym / Fitness Center,Prestige Shantinekatan,Whitefield Road,12.994270,77.729227,"[{'label': 'display', 'lat': 12.99427040849623...",1505,560048,IN,Bangalore,Karnātaka,India,"[Prestige Shantinekatan (Whitefield Road), Ban...",4db41d801e7248d1359a97dc
9,Oliver's,American Restaurant,"Graphite India Road, ITPL Road, Next to Opus",NaN,12.982184,77.708682,"[{'label': 'display', 'lat': 12.98218360116131...",1324,560037,IN,Bangalore,Karnātaka,India,"[Graphite India Road, ITPL Road, Next to Opus,...",511bb096e4b03a144fba17fe


# ***5. Visulization of popular places Nearby***

In [0]:
#Indian Map
map = folium.Map(location=[latitude,longitude],zoom_start=14)

# adding the current location
folium.CircleMarker(location=[latitude,longitude],radius=5,color='red',fill=True,fill_color='red').add_to(map)
folium.Marker(location=[latitude,longitude],popup="<strong> Current Location</strong>",icon = folium.Icon(color='red')).add_to(map)

#drawing circle with radius provided to search
folium.Circle(location=[latitude,longitude],radius=radius,color='blue',opacity=0.5,fill=True,fill_color='blue').add_to(map)
map

In [0]:
for lat,lon,name,cate,addr,dist in zip(nearby.lat,nearby.lng,nearby.name,nearby.categories,nearby.address,nearby.distance):
  folium.CircleMarker(location=[lat,lon],radius=5,color="red",fill=True,fill_color='red').add_to(map)
  folium.Marker(location=[lat,lon],
              popup= folium.Popup(('<strong><font color ="red">Name :     </font>'+name+'</strong><br>'+
                                   '<strong><font color ="red">Categorie :    </font>'+cate+'</strong><br>'+
                                   '<strong><font color ="red">Address :    </font>'+addr+'</strong><br>'+
                                   '<strong><font color ="red">Distance :    </font>'+str(dist)+' mts</strong>'
                                   ),max_width=250)).add_to(map)

map

In [0]:
map.save('PopularPlacesNearMe.html')